In [104]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=30,15
from datetime import datetime

In [ ]:
dataset = pd.read_csv('try.csv')
dataset['Period'] = dataset.apply(lambda row: datetime(int(row['Year']), int(row['Month']), 1) , axis=1)
dataset = dataset.loc[:, ['Period', 'PDSI']]

In [2]:
def test_stationarity(timeseries):
    movingaverage=timeseries.rolling(window=12).mean()
    movingstd=timeseries.rolling(window=12).std()
    orig=plt.plot(timeseries,color='blue')
    mean=plt.plot(movingaverage,color='red')
    dev=plt.plot(movingstd,color='black')
    plt.title('Rolling mean and Standarad Deviation')
    plt.show()

In [102]:
def divide_test(dataset):
    start_date=datetime(2000,1,1)
    mask = (dataset['Period'] >= start_date)
    return dataset.loc[mask]

In [103]:
def divide_training(dataset):
    start_date=datetime(2000,1,1)
    mask = (dataset['Period'] < start_date)
    return dataset.loc[mask]

In [115]:
def model_fit(dataset):
    dataset['Period'] = dataset.apply(lambda row: datetime(int(row['Year']), int(row['Month']), 1) , axis=1)
    dataset = dataset.loc[:, ['Period', 'PDSI']]
    training_set = divide_training(dataset)
    training_set = training_set.set_index(['Period'])
    plt.plot(training_set)
    test_set = divide_test(dataset)
    test_set = test_set.set_index(['Period'])
    
    training_set_shift=training_set-training_set.shift()
    training_set_shift.dropna(inplace=True)
    from statsmodels.tsa.arima_model import ARIMA

    model = ARIMA(training_set_shift,order=(10,1,3))
    results_ARIMA=model.fit(disp=-1)
    predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues,copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA = pd.Series(training_set['PDSI'].ix[0], index=dataset.index)
    predictions_ARIMA = predictions_ARIMA.add(predictions_ARIMA_diff_cumsum,fill_value=0)
    results_ARIMA.plot_predict(1,1000)
    future = results_ARIMA.forecast(steps=test_set.shape[0])
    dataframe = pd.DataFrame(future[0], columns=['PDSI'])
    real_data = test_set
    real_data = real_data.loc[:, ['PDSI']]
    rd = ((real_data - dataframe))/real_data
    size = dataframe.shape[0]
    mape = (sum(rd['PDSI']))/size
    return mape

In [114]:
main_dataset = pd.read_csv('main.csv')
#main_dataset.dropna(inplace=True)
#main_dataset['Period'] = main_dataset.apply(lambda row: datetime(int(row['Year']), int(row['Month']), 1) , axis=1)
main_dataset = main_dataset.loc[:, ['County', 'Year' ,'Month', 'PDSI']]
main_dataset.head()
previous = main_dataset['County'][0]
df = pd.DataFrame(columns=['County', 'Year', 'Month', 'PDSI'])

row = next(main_dataset.iterrows())[1]
row = next(main_dataset.iterrows())[1]
# #df['County'][0]=row['County']
# df.loc[0]=row
# row = next(main_dataset.iterrows())[1]
# df.loc[1]=row
# df

count=0
sum = 0
while(!row.empty):
    if(previous==row['County']):
        df.loc[count]=row
        count=count+1
        row= next(main_dataset.iterrows())[1]
        
    else:
        cur_dataset = df.loc[:, ['County', 'Year' ,'Month', 'PDSI']]
        mape = model_fit(cur_dataset)
        sum = sum + mape
        
mape = sum/count
        
        
    
    
    


False